In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

from openai import OpenAI

load_dotenv()

client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]
)
